In [1]:
import numpy as np
import pandas as pd

In [2]:
source = pd.read_csv('...\\Kepler Exoplanet\\cumulative.csv')

In [3]:
source.head(5)

,rowid,kepid,kepoi_name,kepler_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,1,10797460,K00752.01,Kepler-227 b,CONFIRMED,CANDIDATE,1.000,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,2,10797460,K00752.02,Kepler-227 c,CONFIRMED,CANDIDATE,0.969,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,3,10811496,K00753.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,4,10848459,K00754.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,5,10854555,K00755.01,Kepler-664 b,CONFIRMED,CANDIDATE,1.000,0,0,0,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


In [4]:
del_col = ['rowid', 'kepid', 'kepoi_name', 'kepler_name', 'koi_teq_err1', 'koi_teq_err2',
           'koi_tce_delivname']
data = source

for d in del_col:
    data = data.drop([d], axis=1)

In [5]:
parameter = list(data.columns)
for i in ['koi_disposition', 'koi_pdisposition']: parameter.remove(i)

In [6]:
data = data.fillna(0)

In [7]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
for p in parameter:
    data[p] = pd.Series(scaler.fit_transform(np.array(data[p]).reshape([-1, 1])).reshape([-1]))

C:\Users\user\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [8]:
from sklearn.preprocessing import LabelEncoder

trans = {}
the_label = ['koi_disposition', 'koi_pdisposition']
le = [LabelEncoder(), LabelEncoder()]
for i, l in enumerate(the_label):
    trans[l] = list(pd.DataFrame(data[l].value_counts()).index)
    le[i].fit(trans[l])
    data[l] = le[i].transform(data[l])

In [9]:
trans

{'koi_disposition': ['FALSE POSITIVE', 'CONFIRMED', 'CANDIDATE'],
 'koi_pdisposition': ['FALSE POSITIVE', 'CANDIDATE']}

In [10]:
data_confirmed = data[data.koi_disposition == 1]
data_nonconfirmed = data[data.koi_disposition != 1]

In [11]:
data_confirmed_y = data_confirmed.koi_disposition
data_confirmed_x = data_confirmed.drop(['koi_disposition', 'koi_pdisposition'], axis=1)

data_nonconfirmed_y = data_nonconfirmed.koi_disposition
data_nonconfirmed_x = data_nonconfirmed.drop(['koi_disposition', 'koi_pdisposition'], axis=1)

In [12]:
del_col = ['koi_disposition', 'koi_pdisposition']

for d in del_col:
    data = data.drop([d], axis=1)

In [13]:
from sklearn.model_selection import train_test_split

conf_x_train, conf_x_test, conf_y_train, conf_y_test = train_test_split(data_confirmed_x,
                                                                        data_confirmed_y,
                                                                        test_size=0.3) 
non_x_train, non_x_test, non_y_train, non_y_test = train_test_split(data_nonconfirmed_x,
                                                                    data_nonconfirmed_y,
                                                                    test_size=0.3) 

In [14]:
file = {}

file['x_train'] = pd.concat([conf_x_train, non_x_train])
file['x_test'] = pd.concat([conf_x_test, non_x_test])

file['y_train'] = pd.concat([conf_y_train, non_y_train])
file['y_test'] = pd.concat([conf_y_test, non_y_test])

In [15]:
from sklearn.linear_model import LogisticRegression, SGDClassifier, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

In [16]:
classifier = [LogisticRegression(), SGDClassifier(), LinearRegression(),
              KNeighborsClassifier(), SVC(), GaussianNB(), DecisionTreeClassifier(),
              RandomForestClassifier(), AdaBoostClassifier()]

In [17]:
name = ['Logistic Regression', 'SGD Classifier', 'Linear Regression', 
        'K Neighbors Classifier', 'Support Vector Classifier', 'Gaussian Naive Bayes',
        'Decision Tree Classifier', 'Random Forest Classifier', 'Ada Boost Classifier']

In [18]:
score = np.zeros(len(classifier))

for i, c in enumerate(classifier):
    c.fit(file['x_train'], file['y_train'])
    score[i] = c.score(file['x_test'], file['y_test'])    

C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [19]:
table_1 = pd.concat([pd.DataFrame(name), pd.DataFrame(score)], axis=1)
table_1.columns = ['method', 'accuracy']

In [20]:
from sklearn.preprocessing import OneHotEncoder

ohc = OneHotEncoder()
ohc.fit(np.array(file['y_train']).reshape([-1, 1]))

file['y_train_nn'] = ohc.transform(np.array(file['y_train']).reshape([-1, 1]))
file['y_test_nn'] = ohc.transform(np.array(file['y_test']).reshape([-1, 1]))

In [21]:
from keras.layers import Dense, Dropout
from keras.models import Sequential

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(41, )))
model.add(Dropout(0.25))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(3, activation='softmax'))

C:\Users\user\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [22]:
model.compile(optimizer='rmsprop', metrics=['accuracy'], loss='categorical_crossentropy')

In [23]:
def neural_network(x_train, x_test, y_train, y_test):
    model.fit(x_train, y_train, batch_size=32, epochs=20)
    score = model.evaluate(x_test, y_test, batch_size=32)
    return score

In [24]:
nn_accuracy = neural_network(file['x_train'], file['x_test'], file['y_train_nn'], file['y_test_nn'])

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
6694/6694 [==============================] - 3s 451us/step - loss: 0.4083 - acc: 0.7959
Epoch 2/20
6694/6694 [==============================] - 1s 186us/step - loss: 0.3378 - acc: 0.8266
Epoch 3/20
6694/6694 [==============================] - 1s 165us/step - loss: 0.3215 - acc: 0.8376
Epoch 4/20
6694/6694 [==============================] - 1s 162us/step - loss: 0.3159 - acc: 0.8364
Epoch 5/20
6694/6694 [==============================] - 1s 163us/step - loss: 0.3105 - acc: 0.8487
Epoch 6/20
6694/6694 [==============================] - 1s 162us/step - loss: 0.3086 - acc: 0.8446
Epoch 7/20
6694/6694 [==============================] - 1s 164us/step - loss: 0.3021 - acc: 0.8560
Epoch 8/20
6694/6694 [==============================] - 1s 164us/step - loss: 0.3001 - acc: 0.8552
Epoch 9/20
6694/6694 [==============================] - 1s 164us/step - loss: 0.2962 - acc: 0.8569
Epoch 10/20
6694/6694 [==============================] - 1s 1

In [25]:
nn = {'method':'Neural Network', 'accuracy':[nn_accuracy[1]]}
df_nn = pd.DataFrame().from_dict(nn)

In [26]:
table_1 = pd.concat([table_1, df_nn])
table_1

,accuracy,method
0,0.848780,Logistic Regression
1,0.819164,SGD Classifier
2,0.663308,Linear Regression
3,0.828920,K Neighbors Classifier
4,0.803136,Support Vector Classifier
5,0.784321,Gaussian Naive Bayes
6,0.855052,Decision Tree Classifier
7,0.886063,Random Forest Classifier
8,0.852265,Ada Boost Classifier
0,0.852265,Neural Network


In [27]:
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

fig = go.Figure()
fig.add_trace(go.Bar(x=table_1.method, y=table_1.accuracy))

fig.update_layout(title='The Graph', xaxis_title='Method', yaxis_title='Accuracy')
iplot(fig)

In [28]:
data_confirmed_f = data_confirmed[data_confirmed.koi_pdisposition == 0]
data_confirmed_t = data_confirmed[data_confirmed.koi_pdisposition == 1]

In [29]:
data_confirmed_y_f = data_confirmed_f.koi_pdisposition
data_confirmed_x_f = data_confirmed_f.drop(['koi_disposition', 'koi_pdisposition'], axis=1)

data_confirmed_y_t = data_confirmed_t.koi_pdisposition
data_confirmed_x_t = data_confirmed_t.drop(['koi_disposition', 'koi_pdisposition'], axis=1)

In [30]:
false_x_train, false_x_test, false_y_train, false_y_test = train_test_split(data_confirmed_x_f,
                                                                        data_confirmed_y_f,
                                                                        test_size=0.3) 
true_x_train, true_x_test, true_y_train, true_y_test = train_test_split(data_confirmed_x_t,
                                                                    data_confirmed_y_t,
                                                                    test_size=0.3) 

In [31]:
file_2 = {}

file_2['x_train'] = pd.concat([false_x_train, true_x_train])
file_2['x_test'] = pd.concat([false_x_test, true_x_test])

file_2['y_train'] = pd.concat([false_y_train, true_y_train])
file_2['y_test'] = pd.concat([false_y_test, true_y_test])

In [32]:
score_2 = np.zeros(len(classifier))

for i, c in enumerate(classifier):
    c.fit(file_2['x_train'], file_2['y_train'])
    score_2[i] = c.score(file_2['x_test'], file_2['y_test'])    

C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning:

max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.



In [33]:
table_2 = pd.concat([pd.DataFrame(name), pd.DataFrame(score_2)], axis=1)
table_2.columns = ['method', 'accuracy']

In [34]:
from keras.layers import Dense, Dropout
from keras.models import Sequential

model_2 = Sequential()
model_2.add(Dense(512, activation='relu', input_shape=(41, )))
model_2.add(Dropout(0.25))
model_2.add(Dense(64, activation='relu'))
model_2.add(Dropout(0.25))
model_2.add(Dense(2, activation='softmax'))

In [35]:
model_2.compile(optimizer='rmsprop', metrics=['accuracy'], loss='categorical_crossentropy')

In [36]:
def neural_network_2(x_train, x_test, y_train, y_test):
    model_2.fit(x_train, y_train, batch_size=32, epochs=20)
    score = model_2.evaluate(x_test, y_test, batch_size=32)
    return score

In [37]:
from sklearn.preprocessing import OneHotEncoder

ohc = OneHotEncoder()
ohc.fit(np.array(file_2['y_train']).reshape([-1, 1]))

file_2['y_train_nn'] = ohc.transform(np.array(file_2['y_train']).reshape([-1, 1]))
file_2['y_test_nn'] = ohc.transform(np.array(file_2['y_test']).reshape([-1, 1]))

In [38]:
nn_accuracy = neural_network_2(file_2['x_train'], file_2['x_test'],
                             file_2['y_train_nn'], file_2['y_test_nn'])

Epoch 1/20
1604/1604 [==============================] - 2s 1ms/step - loss: 0.0848 - acc: 0.9782
Epoch 2/20
1604/1604 [==============================] - 0s 292us/step - loss: 0.0326 - acc: 0.9869
Epoch 3/20
1604/1604 [==============================] - 0s 242us/step - loss: 0.0155 - acc: 0.9944
Epoch 4/20
1604/1604 [==============================] - 0s 211us/step - loss: 0.0113 - acc: 0.9950
Epoch 5/20
1604/1604 [==============================] - 0s 209us/step - loss: 0.0104 - acc: 0.9963
Epoch 6/20
1604/1604 [==============================] - 0s 192us/step - loss: 0.0067 - acc: 0.9969
Epoch 7/20
1604/1604 [==============================] - 0s 167us/step - loss: 0.0068 - acc: 0.9975
Epoch 8/20
1604/1604 [==============================] - 0s 162us/step - loss: 0.0069 - acc: 0.9956
Epoch 9/20
1604/1604 [==============================] - 0s 206us/step - loss: 0.0063 - acc: 0.9981
Epoch 10/20
1604/1604 [==============================] - 0s 174us/step - loss: 0.0018 - acc: 1.0000
Epoch 11/20

In [39]:
nn = {'method':'Neural Network', 'accuracy':[nn_accuracy[1]]}
df_nn = pd.DataFrame().from_dict(nn)

In [40]:
table_2 = pd.concat([table_2, df_nn])
table_2

,accuracy,method
0,0.997097,Logistic Regression
1,0.985486,SGD Classifier
2,0.822868,Linear Regression
3,1.000000,K Neighbors Classifier
4,0.981132,Support Vector Classifier
5,0.982583,Gaussian Naive Bayes
6,0.988389,Decision Tree Classifier
7,1.000000,Random Forest Classifier
8,0.997097,Ada Boost Classifier
0,1.000000,Neural Network


In [41]:
fig = go.Figure()
fig.add_trace(go.Bar(x=table_2.method, y=table_2.accuracy))

fig.update_layout(title='The Graph', xaxis_title='Method', yaxis_title='Accuracy')
iplot(fig)